In [1]:
import sys, io
import copy
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint
from itertools import product, permutations, combinations
from typing import List, Optional, Tuple
import random


In [15]:
def F_0(tup):
    a,b,c,S,p = tup
    return (a+1, b+1, (3*c) + (2**(a+p)), S+[a+p],p)
#
def F_1(tup):
    a,b,c,S,p = tup
    return (a+1, b, c, S,p)
#
def F_00(tup):
    return F_0(F_0(tup))
#
def F_01(tup):
    return F_0(F_1(tup))
#
def F_10(tup):
    return F_1(F_0(tup))
#
def F_11(tup):
    return F_1(F_1(tup))
#
def s_Sum(S):
    m = len(S)
    total = 0
    for r in range(m):
        total += ( (3**(m-1-r)) * (2**(S[r])) )
    return total
#
def invF(tup):
    a,b,c,S,p = tup
    if (a == 0):
        return None
    if len(S) > 0:
        max_zero = S[-1]
        if max_zero > a:
            return (a-1, b, c, S,p)
        else:
            S = S[0:-1]
            c = s_Sum(S)            
            return(a-1, b-1, c, S,p)
    else:
        return (a-1, b, c, S,p)
#

In [16]:
tree_root = (0,0,0, [], 0)
tree_root

(0, 0, 0, [], 0)

In [17]:
tup = F_0(tree_root) 
tree_root_0 = (0, tup[1], tup[2], tup[3], 1)
tup = F_1(tree_root) 
tree_root_1 = (0, tup[1], tup[2], tup[3], 1)
[tree_root_0, tree_root_1]

[(0, 1, 1, [0], 1), (0, 0, 0, [], 1)]

In [18]:
def CollatzChain(collatzNumber):
    chain = [collatzNumber]
    while collatzNumber != 1:
        if collatzNumber & 1 == 0:
            collatzNumber = collatzNumber // 2
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
        chain.append(collatzNumber)
    return chain
#

def ChainPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#
def fractionFromNodeTup(tup):
    a, b, c, S, p = tup
    fract = Fraction(2**(a+p) - c, 3**b)
    return (fract.numerator, fract.denominator)
#

def TupChainFromPath(chain_path):
    path_len = len(chain_path)
    p = 0
    tup_chain = [(0, 0, 0, [])]
    if (path_len & 1) == 1:
        p = 1
        
    for chain_item in chain_path:
        p2, p3, c, S = tup_chain[-1]
        if chain_item == "1":
            tup_chain. append((p2 + 1, p3, c, S))
        else:
            tup_chain. append((p2 + 1, p3 + 1, c*3 + 2**p2, S + [p2]))
            
        fract = fractionFromNodeTup(tup_chain[-1])
    return tup_chain
#
def FractionFromPath(chain_path):
    tup = (0, 0, 0)
    for chain_item in chain_path:
        p2, p3, c = tup
        if chain_item == "1":
            tup = (p2 + 1, p3, c)
        else:
            tup = (p2 + 1, p3 + 1, c*3 + 2**p2)
        
    fract = fractionFromNodeTup(tup)
    return fract
#    
def downUpTup(tup):
    p2, p3, c = tup
    p2_01, p3_01, c_01 = (p2 + 2, p3 + 1, c*3 + 2**p2)
    return (p2_01, p3_01, c_01)
#
def TupChain(collatzNumber):
    chain_path = ChainPath(collatzNumber)
    return TupChainFromPath(chain_path)
    return tup_chain
#
def generationTups(n):
    G = [[(0, 0, 0)]]
    for i in range(1, n+1, 1):
        G.append([])
        tups = G[i-1]
        for tup in tups:
            p2, p3, c = tup
            G[i].extend([(p2 + 1, p3, c), (p2 + 1, p3 + 1, c*3 + 2**p2)])
    return G
#
def pathToTup(path):
    a = len(path)
    zeros = [i for i, b in enumerate(path) if b == '0']
    b = len(zeros)
    # compute c = sum_{j=1}^{k} 3^{k-j} * 2^{i_j - 1}
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in enumerate(zeros))
    fract = Fraction(2**a - c, 3**b)
    return path, (a, b, c), (fract.numerator, fract.denominator)
#
def reverseColllatz(collatz_num, generations):
    chains = [[collatz_num]]
    for i in range(generations):
        for chain in chains.copy():
            if ((chain[-1] -1) % 3) == 0:
                reverse_down_val = (chain[-1] - 1)//3
                if (reverse_down_val & 1) == 1:
                    # down only applies to odds
                    chain2 = chain.copy()
                    chain2.append(reverse_down_val)
                    chains.append(chain2)
            chain = chain.append(2*chain[-1])
    #
    return chains
#


def collatzNext(x):
    if x %2 == 0:
        return x//2
    return 3*x + 1
#
def fourPlusOneEdges(maxN):
    for n in range(maxN):
        start = 4*n + 1
        y = collatzNext(start)
        while y % 4 != 1:
            y = collatzNext(y)
        print((n, (y-1)//4))
#
def ToNPlusOneEdges(g, maxN):
    L = []
    multiplier = 2**g
    for n in range(maxN):
        start = multiplier*n + 1
        y = collatzNext(start)
        while y % multiplier != 1:
            y = collatzNext(y)
        L.append((n, (y-1)//multiplier))
    return L
#


def two_factors(n):
    """
    Returns the exponent of the largest power of 2 that divides n.
    For example, if n = 48 (binary 110000), the result is 4 because 2^4 = 16 divides 48.
    """
    if n <= 1:
        return 0 

    count = 0
    while (n & 1) == 0:
        n >>= 1
        count += 1
    return count
#
def gen_generation(a):
    seqs = product('10', repeat=a)
    for bits in seqs:
        label = ''.join(bits)
        zeros = [i for i, b in enumerate(bits) if b == '0']
        b = len(zeros)
        # compute c = sum_{j=0}^{k} 3^{k-j} * 2^{i_j - 1}
        c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in enumerate(zeros))
        f = Fraction(2**a - c, 3**b)
        yield (label, (a,b,c), (f.numerator, f.denominator))
#